In [ ]:
# from nltk.parse.corenlp import CoreNLPParser,CoreNLPDependencyParser
from tqdm.auto import trange, tqdm
from dataclasses import dataclass
import pandas as pd
import ast
import itertools
import wandb
import evaluate
from itertools import cycle
import numpy as np
import random
import time
from datetime import datetime
import collections

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv, TransformerConv, SAGEConv, GraphConv, ResGatedGraphConv, ChebConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from transformers import AutoTokenizer, get_scheduler
from transformers.models.bert.modeling_bert import BertModel

from sklearn.metrics import top_k_accuracy_score

In [ ]:
import transformers
transformers.__version__

'4.18.0'

In [ ]:
import torch_geometric as pyg
pyg.__version__

'2.2.0'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# definitions

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [ ]:
@dataclass
class myGNNoutput:
    loss: None
    logit: None
    emb: None

In [ ]:
def get_loader(df, add_syllables=False, col='pos_seqs', limit=None, batch_size=32, shuffle=True, max_length=128):
    data_list = []
    if limit is not None:
        dfnew = df.sample(frac=1).reset_index(drop=True)[:limit]
    else:
        dfnew = df
    data_list = []
    count = 0
    for i in trange(len(dfnew), leave=False):
        curr = df.iloc[i]
        data = Data()
        data.edge_index = torch.cat([torch.tensor([[0],[0]]),  # for self loop of CLS token
                                     torch.tensor(curr['homo_edges']).T, 
                                     # for batching purpose, if data.x is missing, edge_index is used to inference batch
                                     # an isolated node (the SEP in this case) will mess all up
                                     torch.tensor([[len(curr['homo_edges'])+1],[len(curr['homo_edges'])+1]])], 
                                    axis=1)
        if data.edge_index.shape[1] > max_length:
            count += 1
            continue
        data.text = ' '.join(curr[col])
        data.y = torch.tensor([curr['author']])
        if add_syllables:
            data.num_syllables = torch.tensor([17]+curr['num_syllables']+[17])
            
        if 'doc_id' in curr:
            data.doc_id = torch.tensor([curr['doc_id']])
        data_list.append(data)
    print(f'{count} data dropped because of exceeding max_length {max_length}')
    loader = DataLoader(data_list, batch_size=batch_size, shuffle=shuffle)
    return loader


In [ ]:
GNNtype2layer = {'GCNConv':GCNConv, 
                 'ChebConv':ChebConv, 
                 'SAGEConv':SAGEConv, 
                 'GraphConv':GraphConv,
                 'ResGatedGraphConv':ResGatedGraphConv, 
                 'GATConv':GATConv, 
                 'GATv2Conv':GATv2Conv}

class myHomoGNN(torch.nn.Module):
    def __init__(self, 
                 num_layers, 
                 num_classes, 
                 add_self_loops=False, 
                 gnntype='GCNConv', 
                 add_syllables=False,
                 checkpoint='/scratch/data_jz17d/result/pos_mlm_corenlp/retrained_256_pos_mlm_0/checkpoint-120000/',
                 max_length=256):
        
        super().__init__()
        self.checkpoint = checkpoint
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint, local_files_only=True)
        self.bert = BertModel.from_pretrained(self.checkpoint, local_files_only=True, add_pooling_layer = False).to(device)
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.add_self_loops = add_self_loops
        self.GNNlayer = GNNtype2layer[gnntype]
        self.pos_emb_dim = 32 # this is determined by POS Bert
        
        self.add_syllables = add_syllables
        if add_syllables:
            self.num_syllables = 18 # the longest word has 17 syllables
            self.syllable_emb_layer = nn.Embedding(self.num_syllables, self.pos_emb_dim)
        
        self.gnns = nn.ModuleList()
        for i in range(num_layers):
            if gnntype in ['GCNConv', 'GATConv', 'GATv2Conv']:
                self.gnns.append(self.GNNlayer(self.pos_emb_dim, self.pos_emb_dim, add_self_loops=self.add_self_loops))
            elif gnntype == 'ChebConv':
                self.gnns.append(self.GNNlayer(self.pos_emb_dim, self.pos_emb_dim, K=2)) # hard coded as 2
            else:
                self.gnns.append(self.GNNlayer(self.pos_emb_dim, self.pos_emb_dim))
                
        self.classifier = nn.Linear(self.pos_emb_dim, self.num_classes)
        self.lossfn = nn.CrossEntropyLoss()
        
    def forward(self, text, edge_index, batch, y, ptr, num_syllable=None, readout='pool'):
        tokens = self.tokenizer(text, padding=True, truncation=True, max_length=self.max_length, return_tensors='pt').to(device)
        x = self.bert(**tokens).last_hidden_state
        # reshape! drop padded tokens!
        x = x.masked_select(tokens.attention_mask.ge(0.5).unsqueeze(2)).reshape((-1,self.pos_emb_dim))
        
        if self.add_syllables:
            syllable_emb = self.syllable_emb_layer(num_syllable)
            x = x + syllable_emb
        
        for i in range(self.num_layers):
            x = self.gnns[i](x, edge_index)
            x = F.relu(x)
        
        if readout == 'pool':
            x = global_mean_pool(x, batch) 
        elif readout == 'cls':
            x = x[ptr[:-1],:]
        
        x = F.dropout(x, training=self.training)
        logit = self.classifier(x)
        loss = self.lossfn(logit, y)
        return myGNNoutput(loss=loss, logit=logit, emb=x)

In [ ]:
cols_to_eval = ['homo_edges', 'hetoro_edges', 'pos_seqs', 'upos_seqs', 'num_syllables']

# runs

In [ ]:
max_length = 256

epochs = 100
warmup_ratio = 0.15
monitering_metric = 'accuracy'

LIMIT = [None]
NUM_LAYERS = [3]
LR = [1e-3]
READOUT = ['pool']
GNNTYPE = ['ChebConv']
ADD_SELF_LOOPS = [True]
ADD_SYLLABLES = [True, False]
NUM_SENT = [1,2,3]
REPEAT = list(range(5))

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, READOUT, GNNTYPE, ADD_SELF_LOOPS, ADD_SYLLABLES, NUM_SENT, REPEAT)
num_runs = len(list(ARGS))
run_pbar = trange(num_runs, leave=False)

skip_runs = 7
ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, READOUT, GNNTYPE, ADD_SELF_LOOPS, ADD_SYLLABLES, NUM_SENT, REPEAT)
for i_run, args in enumerate(ARGS):

    if i_run <= skip_runs:
        run_pbar.update(1)
        continue
    limit, num_layers, lr, readout, gnntype, add_self_loops, add_syllables, num_sent_per_text, repeat = args
    
    seed = int(datetime.now().timestamp())
    set_seed(seed)
    
    file = f'../../data/CCAT50/processed/author_all_sent_{num_sent_per_text}_train.csv'
    df = pd.read_csv(file)
    for col in cols_to_eval:
        df[col] = df[col].apply(ast.literal_eval)

    file = f'../../data/CCAT50/processed/author_all_sent_{num_sent_per_text}_val.csv'
    df_val = pd.read_csv(file)
    for col in cols_to_eval:
        df_val[col] = df_val[col].apply(ast.literal_eval)
    val_docid2index = {doc_id:i for i,doc_id in enumerate(df_val['doc_id'].unique())}
    
    valid_loader = get_loader(df_val, add_syllables=add_syllables, max_length=max_length)
    num_valid_steps = len(valid_loader)
    train_loader = get_loader(df, limit = limit, add_syllables=add_syllables, max_length=max_length)
    num_training_steps = len(train_loader)
    
    model = myHomoGNN(num_layers=num_layers,
                      num_classes=50, 
                      add_self_loops=add_self_loops,
                      gnntype=gnntype,
                      add_syllables=add_syllables
                     )
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    wconfig = {}
    wconfig['seed'] = seed
    wconfig['num_sent_per_text'] = num_sent_per_text
    wconfig['limit'] = limit
    wconfig['num_layers'] = num_layers
    wconfig['lr'] = lr
    wconfig['readout'] = readout
    wconfig['GNNtype'] = gnntype
    wconfig['add_self_loops'] = add_self_loops
    wconfig['add_syllables'] = add_syllables
    
    run = wandb.init(project="homo POS GNN (all authors, bert unfrozen)", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run_{i_run}',
                     reinit=True,
                     settings=wandb.Settings(start_method='thread'))
    
    best_evaluation = collections.defaultdict(float)
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            if add_syllables:
                output = model(data.text, data.edge_index, data.batch, data.y, data.ptr, data.num_syllables, readout=readout)
            else:
                output = model(data.text, data.edge_index, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        doc_score = 1e-8*np.ones((len(val_docid2index),50))
        doc_true = np.zeros(len(val_docid2index))
        metric = evaluate.load('/home/jz17d/Desktop/metrics/accuracy')
        for data in valid_loader:
            data.to(device)
            if add_syllables:
                output = model(data.text, data.edge_index, data.batch, data.y, data.ptr, data.num_syllables, readout=readout)
            else:
                output = model(data.text, data.edge_index, data.batch, data.y, data.ptr, readout=readout)
            metric.add_batch(predictions=output.logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
            
            pred = output.logit.argmax(axis=-1).cpu().detach().numpy()
            doc_id = np.vectorize(val_docid2index.get)(data.doc_id.cpu().detach().numpy()) 
            doc_score[doc_id,pred] += 1
            doc_true[doc_id] = data.y.cpu().numpy()
        
        # logging
        evaluation = metric.compute()
        for k in range(1, 6):
            evaluation.update({f'doc_acc@{k}': top_k_accuracy_score(doc_true, doc_score, k=k)})
        wandb.log(evaluation, step=pbar.n)
        
        # logging best
        for key in evaluation:
            best_evaluation[f'best_{key}'] = max(best_evaluation[f'best_{key}'], evaluation[key])
        wandb.log(best_evaluation, step=pbar.n)
    
    run.finish()
    run_pbar.update(1)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/7707 [00:00<?, ?it/s]

0 data dropped because of exceeding max_length 256


  0%|          | 0/30937 [00:00<?, ?it/s]

3 data dropped because of exceeding max_length 256


...

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-

/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/torch_geometric/data/storage.py:280: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y', 'text', 'doc_id'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-

accuracy,▁▂▃▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█▇█████████████████
best_accuracy,▁▂▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███████████████████
best_doc_acc@1,▁▁▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████████████████
best_doc_acc@2,▁▁▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███████████████████
best_doc_acc@3,▁▁▃▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████████
best_doc_acc@4,▁▁▂▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███████████████████
best_doc_acc@5,▁▁▂▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███████████████████
doc_acc@1,▁▁▃▄▃▄▄▅▄▅▅▅▅▆▅▆▆▆▇▇▇▇▇█▇▇▇█▇▇█▇█▇██▇███
doc_acc@2,▁▁▃▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇██▇█████████▇██████
doc_acc@3,▁▁▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█▇▇███████▇█▇██████
doc_acc@4,▁▁▂▄▅▅▅▅▅▆▆▆▆▆▆▇▆▇▇▇▇█▇▇██▇████▇█▇██████
